Reference article for video classification: https://towardsdatascience.com/implementing-real-time-object-detection-system-using-pytorch-and-opencv-70bac41148f7
Reference article github: https://github.com/akash-agni/Real-Time-Object-Detection/blob/main/Object_Detection_Youtube.py

From youtube: https://pypi.org/project/cap-from-youtube/
Opencv display: https://docs.opencv.org/3.4/dd/d43/tutorial_py_video_display.html
Opencv videocapture docs: https://docs.opencv.org/3.4/dd/d9e/classcv_1_1VideoWriter.html

Yolov5 pytorch hub: https://pytorch.org/hub/ultralytics_yolov5/
Yolov5 repo: https://github.com/ultralytics/yolov5/issues/36


In [1]:
import torch
import numpy as np
import cv2 
from cap_from_youtube import cap_from_youtube
from time import time


In [ ]:
class ObjectDetection:
    """
    Class implements Yolo5 model to make inferences on a youtube video using Opencv2.
    """

    def __init__(self, url, out_file="Labeled_Video.avi"):
        """
        Initializes the class with youtube url and output file.
        :param url: Has to be as youtube URL,on which prediction is made.
        :param out_file: A valid output file name.
        """
        self._URL = url
        self.model = self.load_model()
        self.classes = self.model.names
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def get_video_from_url(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :return: opencv2 video capture object, with lowest quality frame available for video.
        """
        play = pafy.new(self._URL).streams[-1]
        assert play is not None
        return cv2.VideoCapture(play.url)

    def load_model(self):
        """
        Loads Yolo5 model from pytorch hub.
        :return: Trained Pytorch model.
        """
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
        labels, cord = results.xyxyn[0][:, -1].numpy(), results.xyxyn[0][:, :-1].numpy()
        return labels, cord

    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """
        return self.classes[int(x)]

    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: void
        """
        player = self.get_video_from_url()
        assert player.isOpened()
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))
        while True:
            start_time = time()
            ret, frame = player.read()
            assert ret
            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)
            end_time = time()
            fps = 1/np.round(end_time - start_time, 3)
            print(f"Frames Per Second : {fps}")
            out.write(frame)


In [ ]:
# Create a new object and execute.
#a = ObjectDetection("https://www.youtube.com/watch?v=dwD1n7N7EAg")
#a()

In [2]:
out_file= "yt_output_video.mp4"

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
classes = model.names

device = 'cuda' if torch.cuda.is_available() else 'cpu'

Using cache found in /Users/kollengruizenga/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-22 Python-3.10.7 torch-2.0.0 CPU

Fusing layers... 


requirements: /Users/kollengruizenga/.cache/torch/hub/requirements.txt not found, check failed.


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
def score_frame(frame):
    """
    Takes a single frame as input, and scores the frame using yolo5 model.
    :param frame: input frame in numpy/list/tuple format.
    :return: Labels and Coordinates of objects detected by model in the frame.
    """
    model.to(device)
    frame = [frame]
    results = model(frame)
    labels, cord = results.xyxyn[0][:, -1].numpy(), results.xyxyn[0][:, :-1].numpy()
    return labels, cord


def plot_boxes(results, frame):
    """
    Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
    :param results: contains labels and coordinates predicted by model on the given frame.
    :param frame: Frame which has been scored.
    :return: Frame with bounding boxes and labels ploted on it.
    """
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]
    for i in range(n):
        row = cord[i]
        if row[4] >= 0.2:
            x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
            bgr = (0, 255, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
            cv2.putText(frame, classes[int(labels[i])], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

    return frame


In [4]:
#Try reading in youtube video
youtube_url = 'https://youtu.be/7HaJArMDKgI'
stream = cap_from_youtube(youtube_url, '720p')

cv2.namedWindow('video_labeled', cv2.WINDOW_NORMAL)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     cv2.imshow('video', frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

[youtube] Extracting URL: https://youtu.be/7HaJArMDKgI
[youtube] 7HaJArMDKgI: Downloading webpage
[youtube] 7HaJArMDKgI: Downloading android player API JSON


In [5]:
#read in from computer camera
#stream = cv2.VideoCapture(0)

if stream.isOpened():
    x_shape = int(stream.get(cv2.CAP_PROP_FRAME_WIDTH))
    y_shape = int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
    four_cc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_file, four_cc, 20, (x_shape, y_shape))
    
    while stream.isOpened(): #while True: 
    #for i in range(25):
        start_time = time()
        ret, frame = stream.read()
        assert ret #True if read frame, raises error if False
        
        results = score_frame(frame)
        frame = plot_boxes(results, frame)
        end_time = time()
        fps = 1/np.round(end_time - start_time, 3) #round to 3 decimals
        print(f"Frames Per Second : {fps}")
        
        # Display the resulting frame
        cv2.imshow('video_labeled', frame)
        
        #Write frame to saved file
        out.write(frame)

        # Press Q on keyboard to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting..")
            stream.release()
            out.release()
            cv2.destroyAllWindows()
            break
    
    stream.release()
    out.release()
    cv2.destroyAllWindows()

Frames Per Second : 3.9215686274509802
Frames Per Second : 5.524861878453039
Frames Per Second : 5.208333333333333
Frames Per Second : 3.676470588235294
Frames Per Second : 5.714285714285714
Frames Per Second : 5.2356020942408374
Frames Per Second : 7.246376811594202
Frames Per Second : 5.555555555555555
Frames Per Second : 5.9171597633136095
Frames Per Second : 4.761904761904762
Frames Per Second : 6.369426751592357
Frames Per Second : 6.666666666666667
Frames Per Second : 6.5359477124183005
Frames Per Second : 6.756756756756757
Frames Per Second : 6.993006993006993
Frames Per Second : 5.813953488372094
Frames Per Second : 5.74712643678161
Frames Per Second : 5.6818181818181825
Frames Per Second : 5.6818181818181825
Frames Per Second : 6.8965517241379315
Frames Per Second : 6.329113924050633
Frames Per Second : 7.142857142857142
Frames Per Second : 6.25
Frames Per Second : 6.5359477124183005
Frames Per Second : 6.849315068493151
Frames Per Second : 6.8965517241379315
Frames Per Second

: 

In [ ]:
stream.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
#np.round(1682208992.947873 - 1682208988.422446, 3)